일반화 성능을 확인하기 위해 반복적으로 테스트 세트를 이용해 성능을 확이하다보면 점점 테스트 세트에 맞추게 된다. <br>
하지만 테스트 세트를 아애 쓰지 않으면 과대/과소적합을 판다하기 어렵다
==> 훈련세트를 또 나누어 쓰자! 

**검증 세트 (validation set)**
- 보통 20~30%를 테스트 세트와 검증 세트로 떼어 놓는다

- Train data : 모델을 학습하는데 사용하는 데이터
- Validation Data : 학습한 모델 성능을 검증하는 데이터
- Test data : 학습한 모델로 예측할 데이터

In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [9]:
from sklearn.model_selection import train_test_split
# train set : 80%, test set : 20%
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 42)
print("train set : {}".format(X_train.shape))
# train set --> sub : 80%, validation set : 20%
X_sub, X_val, y_sub, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
print("train set --> sub : {}".format(X_sub.shape))
print("             & val : {}".format(X_val.shape))

train set : (5197, 3)
train set --> sub : (4157, 3)
             & val : (1040, 3)


In [10]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(X_sub, y_sub)

print(dt.score(X_sub, y_sub))
print(dt.score(X_val, y_val))

0.9971133028626413
0.864423076923077


### 교차 검증 (Cross-Validation)

검증 세트를 만드느라 줄어든 훈련 세트. <br>
보통은 훈련을 할 데이터가 많을 수록 좋은 모델이 만들어진다. <br>
줄어든 훈련 데이터 사이즈에 의한 성능 저하를 개선하기 위해 교차 검증을 사용할 수 있다!

==> **k-fold cross-validation**

In [11]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, X_train, y_train)
print(scores)

# scores는 fit_time, score_time, test_score키를 가진 딕셔너리
# cross_validate함수는 기본적으로 5-fold로 교차검증 수행
# cv 매개변수로 fold조정 가능!

{'fit_time': array([0.00797963, 0.00897884, 0.01296401, 0.01197052, 0.01097417]), 'score_time': array([0.00199318, 0.00099397, 0.00199413, 0.00099397, 0.00298905]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, X_train, y_train, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


splitter (분할기) : 교차검증할 때 훈련 세트를 섞기 위해 사용 <br>
cross_validate함수는 기본적으로 회귀모델일 경우 KFold 분할기를 사용하고, 분류 모델일 경우 StratifiedKFold (계층별 교차 검증)를 사용 <br>
n_splits로 fold수 정하기

In [15]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, X_train, y_train, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


### 평가 방법의 확장
1. 안정적인 일반화 성능 측정 방법인 **교차검증**
2. 

## 하이퍼파라미터 튜닝

In [16]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [17]:
# GridSearchCV는 결정트리 모델의 min_impurity_decrease 값을 바꿔서 총 5번(0.0001 ~ 0.0005) 실행
# cv 기본값은 5, min_impurity_decrease값마다 5-fold cv
# n_jobs 는 병렬 실행에 사용할 CPU 코어 수 지정! (기본값 : 1). -1로 설정하면 시스템에 있는 모든 코어 사용.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

In [18]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [19]:
# 그리드 서치는 훈련이 끝나면 자동으로 가장 검증 점수가 높은 보델의 매개변수 조합으로 전체 훈련 세트에 훈련!
# best_estimator에 저장되어 있다
dt = gs.best_estimator_
print(dt.score(X_train, y_train))

0.9615162593804117


In [20]:
# 최적의 매개변수는 best_params
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [21]:
# 평균 교차 검증 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [22]:
# argmax함수를 사용하면 가장 큰 값의 인덱스를 쉽게 추출할 수 있다
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


좀 더 복잡한 매개변수 조합

In [23]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(X_train, y_train)

print("최상의 매개변수 조합 : ", gs.best_params_)
print("최상의 교차검증 점수 : ", np.max(gs.cv_results_['mean_test_score']))

최상의 매개변수 조합 :  {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
최상의 교차검증 점수 :  0.8683865773302731
